In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy
import os
import nept

In [ ]:
thisdir = os.getcwd()
# data_filepath = os.path.join(thisdir, "cache", "data")
output_filepath = os.path.join(thisdir, "plots", "tmaze")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
rat = "R187"
date = "2019-08-05"
# data_filepath = os.path.join("/home", "emily", "data", rat+"_"+date+"_recording")
# data_filepath = os.path.join("/home", "emily", "data", "RR1_"+date+"_test")

data_filepath = os.path.join("E:\\", "data", "data-experience", rat, "RR3", rat+"_"+date+"_recording")
# data_filepath = os.path.join("E:\\", "data", "data-experience", rat, "RR1_"+date+"_test")

In [ ]:
data_filepath

In [ ]:
import info.r187_tmaze as info

In [ ]:
def get_trials(events):
    if events["trial_start"][0] > events["trial_end"][0]:
        starts = events["trial_end"]
        stops = events["trial_start"]
    else:
        starts = events["trial_start"]
        stops = events["trial_end"]  
        
    if len(starts) > len(stops):
        starts = starts[:-1]
    if len(starts) < len(stops):
        stops = stops[:-1]
    if stops[0]-starts[0] > 0:
        trials = nept.Epoch(starts, stops)
    else:
        trials = nept.Epoch(stops, starts)

    return nept.Epoch(starts, stops)

In [ ]:
event_filename = date+"_Events.nev"
events = nept.load_events(os.path.join(data_filepath, event_filename), info.event_labels)
trials = get_trials(events)
print("Mean:", np.mean(trials.durations), "(s) +/- sem:", scipy.stats.sem(trials.durations))

In [ ]:
evt = nept.load_nev(os.path.join(data_filepath, event_filename))

In [ ]:
evt

In [ ]:
ok = []
for tt in evt:
    ok.append(tt[-1])
np.unique(ok)

In [ ]:
pbs_labels = ["north_pb", "east_pb", "south_pb", "west_pb"]
for pbs_label in pbs_labels:
    print(events[pbs_label])

In [ ]:
events

In [ ]:
spikes = nept.load_spikes(data_filepath)

In [ ]:
n_rows = len(spikes)
add_rows = int(n_rows / 8)

ms = 300 / n_rows
mew = 0.5
spike_loc = 1

fig = plt.figure(figsize=(8, 8))

ax = plt.subplot()
for idx, neuron_spikes in enumerate(spikes):
    ax.plot(neuron_spikes.time, np.ones(len(neuron_spikes.time)) + (idx * spike_loc), '|', color='k', ms=ms, mew=mew)

ax.axis('off')
plt.show()

In [ ]:
lfp = nept.load_lfp(os.path.join(data_filepath, info.lfp_swr_filename))

In [ ]:
plt.plot(lfp.time, lfp.data)
plt.show()

In [ ]:
position_filename = date+"_VT1.nvt"
position = nept.load_position(os.path.join(data_filepath, position_filename), pxl_to_cm=[info.pxl_to_cm["x"], info.pxl_to_cm["y"]])

In [ ]:
plt.plot(position.x, position.y, ".")
plt.show()

In [ ]:
# 720 x 480
plt.plot(position.x, position.time, ".")
plt.show()

In [ ]:
def get_only_tuning_curves(info, position, spikes, epoch_of_interest):
    sliced_position = position.time_slice(epoch_of_interest.starts, epoch_of_interest.stops)
    sliced_spikes = [spiketrain.time_slice(epoch_of_interest.starts, epoch_of_interest.stops) for spiketrain in spikes]

    # Limit position and spikes to only running times
    run_epoch = nept.run_threshold(sliced_position, thresh=10., t_smooth=0.8)
    run_position = sliced_position[run_epoch]
    tuning_spikes = [spiketrain.time_slice(run_epoch.starts, run_epoch.stops) for spiketrain in sliced_spikes]

    tuning_curves = nept.tuning_curve_2d(run_position, tuning_spikes, info.xedges, info.yedges,
                                         occupied_thresh=0.5, gaussian_std=0.3)

    return tuning_curves

In [ ]:
np.where(np.diff(lfp.time) > 1)

In [ ]:
tmaze_times = nept.Epoch(lfp.time[635231], lfp.time[5692603])

In [ ]:
tuning_curves = get_only_tuning_curves(info, position, spikes, tmaze_times)

In [ ]:
xx, yy = np.meshgrid(info.xedges, info.yedges)

cmap = plt.cm.get_cmap('bone_r', 25)

for tuning_curve in tuning_curves:
    plt.figure(figsize=(6, 5))
    plt.plot(position.x, position.y, ".", alpha=0.1, ms=0.3, color="y")
    pp = plt.pcolormesh(xx, yy, tuning_curve, vmin=0.01, cmap=cmap)
    plt.colorbar(pp)
    plt.axis('off')
    plt.tight_layout()
    plt.show()